In [2]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *
from preprocess_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [3]:
tb_fmt = 'png'

In [5]:
for stack in ['MD652', 'MD653']:
    transfer_data_synced(os.path.join('CSHL_data', stack), 
                     from_hostname='s3raw', to_hostname='ec2', is_dir=True, include_only='*.'+tb_fmt)

    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    
    ########### Align ###########
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    script = os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py')
    input_dir = os.path.join(RAW_DATA_DIR, stack)
    output_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
    
    ! rm -r $output_dir
    
    t = time.time()
    print 'Align...'

    run_distributed("%(script)s %(stack)s %(input_dir)s %(output_dir)s \'%%(kwargs_str)s\' %(fmt)s" % \
                    {'script': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir,
                    'fmt': tb_fmt},
                    kwargs_list=[{'prev_fn': valid_filenames[i-1], 'curr_fn': valid_filenames[i]} for i in range(1, len(valid_filenames))],
                    argument_type='list',
                   cluster_size=16,
                   jobs_per_node=16) # 

    wait_qsub_complete()

    print 'done in', time.time() - t, 'seconds' # 252 seconds
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_elastix_output'), 
                     from_hostname='ec2', to_hostname='s3', is_dir=True)

aws s3 cp --recursive s3://mousebrainatlas-rawdata/CSHL_data/MD652 /shared/CSHL_data/MD652 --exclude "*" --include "*.png"


Child returned 0
4.55 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD652
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_sorted_filenames.txt /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
rm: cannot remove '/shared/CSHL_data_processed/MD652/MD652_elastix_output': No such file or directory


Child returned 0
0.51 seconds.
16 nodes requested, 16 nodes available...Continuing


Align...


...

Child returned 0
0.45 seconds.
16 nodes requested, 16 nodes available...Continuing


Align...


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.


done in 397.660309076 seconds
aws s3 cp --recursive /shared/CSHL_data_processed/MD653/MD653_elastix_output s3://mousebrainatlas-data/CSHL_data_processed/MD653/MD653_elastix_output


Child returned 0
21.23 seconds.


- download `elastix_output/` to local machine, edit consecutive transforms in local GUI, generate `custom_transforms/` to S3, upload to S3.
- determine anchor image, upload `anchor.txt` to S3.

In [ ]:
for stack in ['MD652', 'MD653']:
    
    ########### Compose #############
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_custom_transforms'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=True)
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_anchor.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    
    anchor_fn = DataManager.load_anchor_filename(stack=stack)
    
    script = os.path.join(REPO_DIR, 'preprocess', 'compose_transform_thumbnail_v2.py')
    input_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
    output_fn = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                                    dict(stack=stack, anchor_fn=anchor_fn))

    ! rm -f $output_fn
    
    t = time.time()
    print 'Composing transform...'

    run_distributed("%(script)s %(stack)s %(input_dir)s \'%%(kwargs_str)s\' %(anchor_idx)d %(output_fn)s" % \
                {'stack': stack,
                'script': script,
                'input_dir': input_dir,
                'anchor_idx': valid_filenames.index(anchor_fn),
                'output_fn': output_fn},
                kwargs_list=[{'filenames': valid_filenames}],
                argument_type='list',
                   cluster_size=1,
                   exclude_nodes=[33])

    linked_name = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack))
    execute_command('rm -f ' + linked_name)
    execute_command('ln -s ' + output_fn + ' ' + linked_name)

    print 'done in', time.time() - t, 'seconds'
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack), 
                     from_hostname='ec2', to_hostname='s3', is_dir=True, include_only='*.pkl')
    
    ########## Warp #############
    
    if stack in all_nissl_stacks:
        pad_bg_color = 'white'
    else:
        pad_bg_color = 'auto'
        
    script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')
    input_dir = os.path.join(RAW_DATA_DIR, stack)
    out_dir = os.path.join(DATA_DIR, stack, stack + '_thumbnail_alignedTo_' + anchor_fn)
    
    ! rm -rf $out_dir
    
    t = time.time()
    print 'Warping...'

    transforms_filename = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                       dict(stack=stack, anchor_fn=anchor_fn))
    transforms_to_anchor = pickle.load(open(transforms_filename, 'r'))

    if pad_bg_color == 'auto':
        run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                        {'script': script,
                        'stack': stack,
                        'input_dir': input_dir,
                        'out_dir': out_dir
                        },
                        kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                    'filename': fn + '.' + tb_fmt,
                                    'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif',
                                    'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                    for fn in valid_filenames],
                        exclude_nodes=[33],
                        argument_type='single',
                       cluster_size=16,
                       jobs_per_node=8)
    else:
        run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %(pad_bg_color)s' % \
                        {'script': script,
                        'stack': stack,
                        'input_dir': input_dir,
                        'out_dir': out_dir,
                        'pad_bg_color': pad_bg_color},
                        kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                    'filename': fn + '.' + tb_fmt,
                                    'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif'}
                                    for fn in valid_filenames],
                        exclude_nodes=[33],
                        argument_type='single',
                       cluster_size=16,
                       jobs_per_node=8)
        
    wait_qsub_complete()

    print 'done in', time.time() - t, 'seconds'
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_thumbnail_alignedTo_' + anchor_fn),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)